## About data set

There are 3 of datasets that we will use for the analysis:

1. order_dataset
order_id
* customer_id
* order_status
* order_purchase_timestamp
* order_approved_at
* order_delivered_carrier_date
* order_delivered_customer_date
* order_estimated_delivery_date

2. customers_dataset
* customer_id
* customer_unique_id
* customer_zip_code_prefix
* customer_city
* customer_state

3. order_payment_dataset
* order_id
* payment_sequential
* payment_type
* payment_installments
* payment_value